[ 임베딩 모델 - Skip-Gram ]
- 중심 단어를 입력받아 주변 단어 예측하는 모델 
- 중심 단어를 기준으로 양쪽 윈도 크기 만큼 단어들을 주변 단어로 삼아 훈련 데이터 생성
- CBoW 보다 더 뛰어난 성능
- 비교적 드물게 등장하는 단어를 더 잘 학습


- 입력 : 중심 단어의 원-핫 벡터.
- 출력 : 주변 단어들에 대한 확률 분포.





In [1]:
# 기본 Skip-gram 클래스 
from torch import nn

class VanillaSkipgram(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(num_embeddings=vocab_size,     # 임베딩할 수 = 단어사전의 사이즈 
                                      embedding_dim=embedding_dim)   # 임베딩 벡터의 차원
        
        self.linear = nn.Linear(in_features=embedding_dim,
                                out_features=vocab_size)
        
        
    def forward(self, input_ids):
        embedding = self.embedding(input_ids)
        output = self.linear(embedding)
        
        return output 
        


In [2]:
# 영화 리뷰 데이터세트 전처리
import pandas as pd
from Korpora import Korpora
from konlpy.tag import Okt

corpus = Korpora.load('nsmc')
corpus = pd.DataFrame(corpus.test)

tokenizer = Okt()     # 한국어 형태소 분석기 

# 형태소 분석기.morphs() : 텍스트를 형태소 단위로 나누어 문자열 리스트로 반환
tokens = [tokenizer.morphs(review) for review in corpus.text]


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\knudc\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\knu

In [3]:
# 단어 사전 구축
from collections import Counter     # 빈도수 확인을 위한 클래스 


def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    
    for tokens in corpus:
        # update():리스트나 튜플의 항목들을 추가하여, 각 항목의 빈도를 누적하여 업데이트
        counter.update(tokens)
    
    vocab = special_tokens
    
   
	# counter.most_common(n_vocab) : Counter 객체의 가장 많이 등장한 항목을 빈도수 순으로 반환.
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
        
    return vocab

# 단어 사전 만드는 코드 
vocab = build_vocab(corpus=tokens, n_vocab=5000, special_tokens=["<unk>"])

token_to_id = {token:idx for idx, token in enumerate(vocab)}
id_to_token = {idx:token for idx, token in enumerate(vocab)}

In [4]:
# 중심 단어와 문맥 단어를 단어쌍으로 추출 
def get_word_pairs(tokens, window_size):
    pairs = []
    
    for sentence in tokens:
        sentence_length = len(sentence)
        
        for idx, center_word in enumerate(sentence):
            window_start = max(0, idx - window_size)
            window_end = min(sentence_length, idx + window_size + 1)
            center_word = sentence[idx]
            context_word = sentence[window_start:idx] + sentence[idx+1:window_end]

            for context_word in context_word:
                pairs.append([center_word, context_word])
                
    return pairs

In [5]:
word_pairs = get_word_pairs(tokens, window_size=2)
print(word_pairs[:5])

[['굳', 'ㅋ'], ['ㅋ', '굳'], ['뭐', '야'], ['뭐', '이'], ['야', '뭐']]


In [6]:
# 단어 쌍을 인덱스 쌍으로 변환
def get_index_pairs(word_pairs, token_to_id):
    pairs = []
    
    unk_index = token_to_id['<unk>']
    
    for word_pair in word_pairs:
        center_word, context_word = word_pair
        center_index = token_to_id.get(context_word, unk_index)
        pairs.append([center_index, center_index])
    
    return pairs
    

In [7]:
index_pairs = get_index_pairs(word_pairs, token_to_id)
print(index_pairs[:5])

[[100, 100], [595, 595], [176, 176], [2, 2], [77, 77]]


In [8]:
# 데이터 로더 적용 

import torch
from torch.utils.data import TensorDataset, DataLoader


index_pairs = torch.tensor(index_pairs)
center_indexes = index_pairs[:,0]
context_indexes = index_pairs[:,1]
dataset = TensorDataset(center_indexes, context_indexes)
dataLoader = DataLoader(dataset, batch_size=32, shuffle=True)

In [9]:
#  Skip-gram 모델 준비작업
from torch import optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

word2vec = VanillaSkipgram(vocab_size=len(token_to_id), embedding_dim=128).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(word2vec.parameters(), lr=0.1)

In [10]:
# 모델 학습
for epoch in range(10):
    cost = 0.0
    
    for input_ids, target_ids in dataLoader:
        input_ids = input_ids.to(device)
        target_ids = target_ids.to(device)
        
        logits = word2vec(input_ids)
        
        loss = criterion(logits, target_ids)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        cost += loss
        
    cost = cost/ len(dataLoader)
    
    print(f'Epoch :{epoch+1:4d}, Cost: {cost:.3f}')

Epoch :   1, Cost: 0.236


KeyboardInterrupt: 

In [ ]:
# 임베딩 값 추출
token_to_embedding = dict()

embedding_matrix = word2vec.embedding.weight.detach().cpu().numpy()

for word, embedding in zip(vocab, embedding_matrix):
    token_to_embedding[word] = embedding
    
    
index = 30
token = vocab[index]
token_embedding = token_to_embedding[token]
print(token)
print(token_embedding)

연기
[ 0.03148203 -1.7430369  -0.9985893   0.35697037 -0.5312408   0.8770849
  1.652369   -1.216072    0.26213446 -2.0138936  -1.4600477  -0.6476084
  0.20338486  0.5522846  -1.4549627  -0.12665996 -0.30877173  1.3791674
 -1.4885144  -0.7656742   0.04201519 -1.3023247   1.2809416   2.294082
  0.3020047  -0.76034963 -0.78962076 -0.892265    1.0407195  -0.39469522
 -0.06406677 -1.2985444  -0.33926186 -1.3219382   0.04731671  0.33967048
  0.3444855  -1.8515513   0.588643    0.31204346  1.0649177  -0.30699804
  1.1828293  -0.40092054 -1.2336591   0.0139327   1.4040414   1.8865713
 -0.772403   -0.30176172 -0.03377921  1.7559062  -1.0170192  -1.7649146
 -0.7136461  -0.16428049  0.19048955  1.6494552  -0.16430807  0.05979941
  0.5634676  -1.1211002  -0.46502906  0.40692717  2.3519185  -0.8555412
 -0.66843224 -0.7770644   0.2735323   0.05094188 -0.77083945 -1.454007
  0.173752   -0.59047174 -0.1688112  -1.1925869   1.139278   -2.0258534
 -0.134264   -0.84478074 -4.172071    0.54305995 -1.3474511

In [ ]:
# 단어 임베딩 유사도 계산
import numpy as np
from numpy.linalg import norm


def cosine_similarity(a,b):
    cosine = np.dot(b,a) / (norm(b,axis=1) * norm(a))
    
    return cosine


def top_n_index(cosine_matrix,n):
    closest_indexes = cosine_matrix.argsort()[::-1]
    top_n = closest_indexes[1:n+1]
    
    return top_n


cosine_matrix = cosine_similarity(token_embedding, embedding_matrix)
top_n = top_n_index(cosine_matrix, n=5)

print(f'{token}과 가장 유사한 5개 단어')
for index in top_n:
    print(f'{id_to_token[index]} - 유사도 : {cosine_matrix[index]:.4f}')
